In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, KFold

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve

from sklearn.metrics import accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix


from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import pickle
from sklearn.metrics import ConfusionMatrixDisplay
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import warnings
import pickle
warnings.filterwarnings('ignore')
%load_ext autotime

time: 0 ns (started: 2021-12-14 15:21:27 +01:00)


In [2]:
def evaluate_model(ytest, ypred, ypred_proba = None):
    if ypred_proba is not None:
        print('ROC-AUC score of the model: {}'.format(roc_auc_score(ytest, ypred_proba[:, 1])))
    print('Accuracy of the model: {}\n'.format(accuracy_score(ytest, ypred)))
    print('Classification report: \n{}\n'.format(classification_report(ytest, ypred)))

time: 0 ns (started: 2021-12-14 15:21:27 +01:00)


In [3]:
xtrain = pd.read_parquet("../data/xtrain.parquet")
ytrain = pd.read_parquet("../data/ytrain.parquet")['fatality']
xtest = pd.read_parquet("../data/xtest.parquet")
ytest = pd.read_parquet("../data/ytest.parquet")['fatality']

time: 438 ms (started: 2021-12-14 15:21:27 +01:00)


In [4]:
### Parameter Tunning Optimization
param_grid = { 
    'is_unbalance': [True],
    'max_depth': [-1],
    'objective': ['binary'],
    'learning_rate': [0.1, 0.05], 
    'n_estimators' : [200, 300, 400], 
    'importance_type' :['split', 'gain'],
    'num_leaves':[50, 100, 200]
    }

CV = GridSearchCV(lgb.LGBMClassifier(), param_grid, cv=5, n_jobs=-1, scoring='roc_auc')
CV.fit(xtrain, ytrain)

GridSearchCV(cv=5, estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'importance_type': ['split', 'gain'],
                         'is_unbalance': [True], 'learning_rate': [0.1, 0.05],
                         'max_depth': [-1], 'n_estimators': [200, 300, 400],
                         'num_leaves': [50, 100, 200],
                         'objective': ['binary']},
             scoring='roc_auc')

time: 52min 8s (started: 2021-12-14 15:21:28 +01:00)


In [5]:
with open('../models/LightGBM_pipeline.pickle', 'wb') as f:
    pickle.dump(CV, f)

time: 141 ms (started: 2021-12-14 16:13:36 +01:00)


In [6]:
# Para no tener que ejecutar, saltarse el fit y ejecutar a partir de aquí
with open('../models/LightGBM_pipeline.pickle', 'rb') as f:
    CV = pickle.load(f)

time: 32 ms (started: 2021-12-14 16:13:36 +01:00)


In [7]:
print(CV.best_params_)
print(CV.best_score_)
ypred = CV.predict(xtest)
ypred_proba = CV.predict_proba(xtest)

{'importance_type': 'split', 'is_unbalance': True, 'learning_rate': 0.05, 'max_depth': -1, 'n_estimators': 300, 'num_leaves': 200, 'objective': 'binary'}
0.8578480278752464
time: 8.03 s (started: 2021-12-14 16:13:36 +01:00)


In [8]:
evaluate_model(ytest, ypred, ypred_proba)

ROC-AUC score of the model: 0.8607598814863755
Accuracy of the model: 0.802971651183402

Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.80      0.89    797650
           1       0.06      0.74      0.10     12472

    accuracy                           0.80    810122
   macro avg       0.53      0.77      0.50    810122
weighted avg       0.98      0.80      0.88    810122


time: 1.27 s (started: 2021-12-14 16:13:44 +01:00)


In [9]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)

Best Threshold=0.443154, G-Mean=0.775
ROC-AUC score of the model: 0.8607598814863755
Accuracy of the model: 0.7583796020846243

Classification report: 
              precision    recall  f1-score   support

           0       1.00      0.76      0.86    797650
           1       0.05      0.79      0.09     12472

    accuracy                           0.76    810122
   macro avg       0.52      0.77      0.48    810122
weighted avg       0.98      0.76      0.85    810122


time: 1.45 s (started: 2021-12-14 16:13:46 +01:00)


In [10]:
### Parameter Tunning Optimization
param_grid = { 
    'n_estimators': [200, 250, 500],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [4, 6, 8, None],
    'criterion' :['gini'],
    'class_weight':['balanced']}


CV = RandomizedSearchCV(RandomForestClassifier(), param_grid, cv=5, random_state=0, n_jobs=-1, scoring='roc_auc')
                  
CV.fit(xtrain, ytrain)
print(CV.best_params_)
print(CV.best_score_)

{'n_estimators': 250, 'max_features': 'log2', 'max_depth': 6, 'criterion': 'gini', 'class_weight': 'balanced'}
0.8158191758291675
time: 2h 30min 37s (started: 2021-12-14 16:13:47 +01:00)


In [11]:
with open('../models/RandomForest_pipeline.pickle', 'wb') as f:
    pickle.dump(CV, f)

time: 47 ms (started: 2021-12-14 18:44:25 +01:00)


In [12]:
# Para no tener que ejecutar, saltarse el fit y ejecutar a partir de aquí
with open('../models/RandomForest_pipeline.pickle', 'rb') as f:
    CV = pickle.load(f)

time: 16 ms (started: 2021-12-14 18:44:25 +01:00)


In [13]:
print(CV.best_params_)
print(CV.best_score_)
ypred = CV.predict(xtest)
ypred_proba = CV.predict_proba(xtest)

{'n_estimators': 250, 'max_features': 'log2', 'max_depth': 6, 'criterion': 'gini', 'class_weight': 'balanced'}
0.8158191758291675
time: 29.4 s (started: 2021-12-14 18:44:25 +01:00)


In [14]:
evaluate_model(ytest, ypred, ypred_proba)

ROC-AUC score of the model: 0.8149439139334367
Accuracy of the model: 0.7290654987767274

Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.73      0.84    797650
           1       0.04      0.74      0.08     12472

    accuracy                           0.73    810122
   macro avg       0.52      0.74      0.46    810122
weighted avg       0.98      0.73      0.83    810122


time: 1.33 s (started: 2021-12-14 18:44:54 +01:00)


In [15]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)

Best Threshold=0.513827, G-Mean=0.737
ROC-AUC score of the model: 0.8149439139334367
Accuracy of the model: 0.7478935271477629

Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.75      0.85    797650
           1       0.04      0.73      0.08     12472

    accuracy                           0.75    810122
   macro avg       0.52      0.74      0.47    810122
weighted avg       0.98      0.75      0.84    810122


time: 1.44 s (started: 2021-12-14 18:44:56 +01:00)
